In [17]:
from src.features.videos.objects import object as object_helper
from src import constants
import psycopg2

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Get the sum of all detections per video with probability > 50%
yolo_results = object_helper.get_probability_sum(model="yolo")
tiny_yolo_results = object_helper.get_probability_sum(model="tiny_yolo")

In [24]:
# Get the total number of seconds analyzed for each of the classifiers
conn = psycopg2.connect(database="video_article_retrieval", user="postgres")
c = conn.cursor()
c.execute("SELECT count(DISTINCT (id, second)) FROM object_detection_yolo")
yolo_seconds, = c.fetchone()
c.execute("SELECT count(DISTINCT (id, second)) FROM object_detection_tiny_yolo")
tiny_yolo_seconds, = c.fetchone()

In [29]:
# Get rate of detections per second across all videos
cols = [label.replace(" ", "_") for label in constants.COCO_CLASS_NAMES]
cols = ["traffic_light"]
yolo_results[cols].sum().sum() / yolo_seconds, tiny_yolo_results[cols].sum().sum() / tiny_yolo_seconds

(0.014044165305744611, 0.00221237208203292)